In [1]:
# Load the environment variables
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("env_vars.env")) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Hugging Face Hub LLM

In [2]:
from langchain import PromptTemplate
from langchain import HuggingFaceHub, LLMChain

template = """Question: {question} Answer: """
prompt = PromptTemplate(template=template, input_variables=['question'])
print(prompt)

# Initialize Hub LLM
hub_llm = HuggingFaceHub(repo_id='google/flan-t5-xl', model_kwargs={'temperature':1e-10})

# Create prompt template > LLM chain
llm_chain = LLMChain(prompt=prompt, llm=hub_llm)

input_variables=['question'] output_parser=None partial_variables={} template='Question: {question} Answer: ' template_format='f-string' validate_template=True


/Users/trucvietle/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Ask the question
question = "Who is the richest person in the world?"
print(question)

# NOTE: This will likely time out!
# print(llm_chain.run(question))

Who is the richest person in the world?


# OpenAI LLM

In [4]:
from langchain.llms import OpenAI

# Initialize the model
davinci = OpenAI(model_name="text-davinci-003")
llm_chain = LLMChain(prompt=prompt, llm=davinci)

In [5]:
# Ask the question
print(llm_chain.run(question))



The richest person in the world as of October 2020 is Amazon founder Jeff Bezos, with a net worth of approximately $182 billion.


# Prompt Templates

Two kinds of knowledge that can be "reasoned" by LLM:
- **Parametric knowlege** -- learned during training and stored in model parameters (weights)
- **Source knowledge** -- provided to the model at inference time as contexts and examples (via input prompt)

In [6]:
prompt = """The following is a conversation with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: User: How to solve a Rubik's Cube? AI: """

openai.temperature = 1.0 # increase creativity/randomness of output
print(davinci(prompt))



"That's easy. All you have to do is find someone who already knows how to solve it and then copy them!"


In [7]:
# To help the model, we can give it a few examples of the type of answers we’d like
prompt = """The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: User: How are you? AI: I can't complain but sometimes I still do. 
User: What time is it? AI: It's time to get a watch. User: How to solve a matrix rotation problem? AI: """

print(davinci(prompt))


Well, that's a bit out of my area of expertise - I'm more of a conversational AI. But I'm sure you'll figure it out!


In [32]:
# FewShotPromptTemplate caters to source knowledge input
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, 
    {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """ User: {query} AI: {answer} """

In [34]:
# create a prompt example from above template
example_prompt = PromptTemplate(input_variables=["query", "answer"], template=example_template)
print(example_prompt)

input_variables=['query', 'answer'] output_parser=None partial_variables={} template=' User: {query} AI: {answer} ' template_format='f-string' validate_template=True


In [35]:
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions (contexts)
prefix = """The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: """

# and the suffix our user input and output indicator
suffix = """ User: {query} AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(examples=examples, example_prompt=example_prompt,
                                                 prefix=prefix, suffix=suffix, input_variables=["query"],
                                                 example_separator="\n\n")
print(few_shot_prompt_template)

input_variables=['query'] output_parser=None partial_variables={} examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}] example_selector=None example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True) suffix=' User: {query} AI: ' example_separator='\n\n' prefix='The following are exerpts from conversations with an AI assistant. \nThe assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. \nHere are some examples: ' template_format='f-string' validate_template=True


In [36]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: 

 User: How are you? AI: I can't complain but sometimes I still do. 

 User: What time is it? AI: It's time to get a watch. 

 User: What is the meaning of life? AI: 


In [37]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [38]:
print(examples)

[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}]


In [39]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(examples=examples,
                                              example_prompt=example_prompt,
                                              max_length=50 # this sets the max length that examples should be
                                             )
print(example_selector)

examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}] example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True) get_text_length=<function _get_length_based at 0x11b59e3b0> max_length=50 example_text_lengths=[15, 14, 11, 16, 10, 19, 19]


In [40]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector, # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)
print(dynamic_prompt_template)

input_variables=['query'] output_parser=None partial_variables={} examples=None example_selector=LengthBasedExampleSelector(examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}], example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True), get_text_length=<function _get_length_based at 0x11b59e3b0>, max_length=50,

In [43]:
prompt = dynamic_prompt_template.format(query="How do birds fly?")
print(prompt)

The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: 
 User: How are you? AI: I can't complain but sometimes I still do. 
 User: What time is it? AI: It's time to get a watch. 
 User: What is the meaning of life? AI: 42 
 User: How do birds fly? AI: 


In [44]:
print(davinci(prompt))

 With a little help from their friends!


# Memory

In [47]:
from langchain import OpenAI
from langchain.chains import ConversationChain

# first initialize the large language model
llm = OpenAI(temperature=0, model_name="text-davinci-003")

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## ConversationBufferMemory

In [48]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(llm=llm, memory=ConversationBufferMemory())
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today, isn't it? How can I help you?"}

In [49]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [50]:
count_tokens(conversation_buf,
             "My interest here is to explore the potential of integrating Large Language Models with external knowledge")

Spent a total of 178 tokens


' Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?'

In [51]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI:  Good morning! It's a beautiful day today, isn't it? How can I help you?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?


## ConversationSummaryMemory

In [52]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(llm=llm, memory=ConversationSummaryMemory(llm=llm))
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [53]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(conversation_sum, "Good morning AI!")

Spent a total of 283 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [55]:
count_tokens(conversation_sum, 
             "My interest here is to get filthy rich and live a lavish life!"
            )

Spent a total of 383 tokens


' That sounds like an exciting goal! What kind of steps do you think you need to take to achieve it?'

In [56]:
count_tokens(conversation_sum, 
             "I just want to analyze the different possibilities to that goal. What can you think of?"
            )

Spent a total of 539 tokens


" That's a great goal! I think the first step is to identify what resources you have available to you. Do you have any savings, investments, or other assets that you can use to help you reach your goal?"

In [57]:
print(conversation_sum.memory.buffer)


The human greeted the AI and the AI responded with a greeting and asked how it could help. The human expressed their goal of getting rich and living a lavish life, to which the AI responded positively and asked what steps the human thought they needed to take to achieve it. The human asked the AI to analyze the different possibilities to that goal, and the AI suggested the first step should be to identify what resources the human had available to them.


## ConversationBufferWindowMemory

In [63]:
# Acts in the same way as our earlier “buffer memory” but adds a window to the memory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# We set k=1 -- meaning the window will remember the single latest interaction between the human and AI
conversation_bufw = ConversationChain(llm=llm, memory=ConversationBufferWindowMemory(k=1))

In [64]:
bufw_history = conversation_bufw.memory.load_memory_variables(inputs=[])['history']
print(bufw_history)

## ConversationSummaryBufferMemory

In [67]:
# Summarizes the earliest interactions in a conversation while maintaining the most recent tokens in the conversation
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

conversation_sum_bufw = ConversationChain(llm=llm, 
                                          memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=650)
                                         )

This the only one of our memory types (so far) that allows us to remember distant interactions and store the most recent interactions in their raw form.

In [68]:
# Others:
# Knowledge graph
from langchain.chains.conversation.memory import ConversationKGMemory

# Entities -- remember details about the entities (persons, etc.)
from langchain.chains.conversation.memory import ConversationEntityMemory

# Knowledge Base

The external knowledge base is the "window" into the world beyond the LLM’s training data.

In [71]:
from datasets import load_dataset # from Hugging Face

data = load_dataset("wikipedia", "20220301.simple", split='train[:10000]')
data

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1.66k/1.66k [00:00<00:00, 2.20MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 235M/235M [00:22<00:00, 10.4MB/s]


Dataset wikipedia downloaded and prepared to /Users/trucvietle/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 10000
})

In [83]:
data[36]

{'id': '63',
 'url': 'https://simple.wikipedia.org/wiki/Acceleration',
 'title': 'Acceleration',
 'text': 'Acceleration is a measure of how fast velocity changes. Acceleration is the change of velocity divided by the change of time. Acceleration is a vector, and therefore includes both a size and a direction. Acceleration is also a change in speed and direction, there is:\n\nSpeed (a scalar quantity) (uses no direction)\n\n Distance is how far you traveled \n Time is how long it took you to travel \n Speed is how fast you are moving - Speed = Distance / Time \n\nVelocity (a vector quantity) (uses a direction)\n\n Displacement is how much your position has changed in what direction\n Velocity is how quickly your position is changing and in what direction\n Velocity = Displacement / Time \n\nThe measurement of how fast acceleration changes is called jerk.\n\nExamples \n An object was moving north at 10 meters per second. The object speeds up and now is moving north at 17 meters per secon

## Create chunks

A token is typically the size of a word or sub-word and varies by LLM. The tokens themselves are built using a *tokenizer*.

In [79]:
import tiktoken

tokenizer = tiktoken.get_encoding("p50k_base")

# create the length function
def tiktoken_len(text):
    
    tokens = tokenizer.encode(text, disallowed_special=())
    
    return len(tokens)

text = "hello I am a chunk of text and using the tiktoken_len function \
we can find the length of this chunk of text in tokens"

print(tiktoken_len(text))
print(len(text))

28
117


In [82]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20,
                                               length_function=tiktoken_len,
                                               separators=["\n\n", "\n", " ", ""])

chunks = text_splitter.split_text(data[36]['text'])[:3]
chunks

["Acceleration is a measure of how fast velocity changes. Acceleration is the change of velocity divided by the change of time. Acceleration is a vector, and therefore includes both a size and a direction. Acceleration is also a change in speed and direction, there is:\n\nSpeed (a scalar quantity) (uses no direction)\n\n Distance is how far you traveled \n Time is how long it took you to travel \n Speed is how fast you are moving - Speed = Distance / Time \n\nVelocity (a vector quantity) (uses a direction)\n\n Displacement is how much your position has changed in what direction\n Velocity is how quickly your position is changing and in what direction\n Velocity = Displacement / Time \n\nThe measurement of how fast acceleration changes is called jerk.\n\nExamples \n An object was moving north at 10 meters per second. The object speeds up and now is moving north at 17 meters per second. The object has accelerated. \n An apple is falling down. It starts falling at 0 meters per second. At 

In [87]:
print(len(chunks))
tiktoken_len(chunks[0]), tiktoken_len(chunks[1]), tiktoken_len(chunks[2])

3


(374, 392, 48)

## Create embeddings

In [93]:
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# model_name = 'text-embedding-ada-002'

# embed = OpenAIEmbeddings()
embed = HuggingFaceEmbeddings()

In [94]:
texts = ['this is the first chunk of text',
         'then another second chunk of text is here']

res = embed.embed_documents(texts)
len(res), len(res[0])

(2, 768)

## Vectorstore

In [95]:
from langchain.vectorstores import Chroma

persist_directory = '../docs/chroma/'

!rm -rf ./docs/chroma  # remove old database files if any

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [113]:
from tqdm.auto import tqdm

batch_limit = 100
texts = []
metadatas = []

for i, record in enumerate(tqdm(data)):
    # first get metadata fields for this record
    metadata = {
        'wiki-id': str(record['id']),
        'source': record['url'],
        'title': record['title']
    }
    
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['text'])
                                                 
    # create individual metadata dicts for each chunk
    record_metadatas = [{"chunk": j, "text": text, **metadata} for j, text in enumerate(record_texts)]
    
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    
#     # if we have reached the batch_limit we can add texts
#     if len(texts) >= batch_limit:
#         embeds = embed.embed_documents(texts)
        
#         vectordb = Chroma.from_documents(
#             documents=texts,
#             embedding=embeds,
#             persist_directory=persist_directory
#         )
        
#         texts = []
#         metadatas = []

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:08<00:00, 1234.48it/s]


## Query vectorstore

In [120]:
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embed)
print(vectorstore._collection.count())

12


In [123]:
query = "What is the minimum spend?"

vectorstore.similarity_search(query, # our search query
                              k=3 # return 3 most relevant docs)

[Document(page_content='(i) “Electric Vehicle Charging” is defined as Card Transactions classified under the MCC 5552  \n(Electric Vehicle Charging).  \n \nB. 0.25% cashback on the following transactions:  \n• all Card Transactions if the Minimum Spend Requirement is not met,  \n• all Card Transaction s (excluding Card Transactions under the categories listed at Clause \n2A) if the Minimum Spend Requirement is met; and  \n• Card Transactions expressly stated in Clause 2A(a)(iii) above.  \n  \n3. Minimum Spend Requirement  \n \na) The Minimum Spend Requirement must be met to receive the Cashback listed at Clause 2A \nabove. There is no minimum spend requirement to receive the Base Cashback. The Minimum \nSpend Requirement can be aggregated between the Principal and Supplementary Cardmembers. \nThe maximum amount of Cashback t hat one account (the Principal and all Supplementary \nCardmembers together) can earn in any calendar month is S$160.  \n \nMinimum Spend \nRequirement  Cashback C

## Generative QA

In [124]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
qa.run(query)

'The minimum spend requirement is the amount of money that needs to be spent on the OCBC 365 Card in order to qualify for certain cashback rewards. The specific amount of the minimum spend requirement is not mentioned in the given context.'

In [125]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="stuff",
                                                              retriever=vectorstore.as_retriever()
                                                             )

query = "Are there any dining rewards?"
qa_with_sources(query)

{'question': 'Are there any dining rewards?',
 'answer': 'Yes, there are dining rewards. The OCBC 365 Card offers 5% cashback on "Dining" card transactions, which includes transactions made at restaurants, cafes, caterers, and fast food restaurants. However, transactions made at hotels, wedding banquets, and transactions not made in person (such as telephone or mail orders) are not eligible for the 5% cashback. \n',
 'sources': '../docs/tncs-365cc-programme.pdf'}

# Agents

We can think of agents as enabling “tools” for LLMs. Like how a human would use a calculator for maths or perform a Google search for information — agents allow an LLM to do the same thing.

Agents are LLMs that can use tools like calculators, search, or executing code.

In [135]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm = OpenAI(temperature=0, model_name="text-davinci-003")
llm_math = LLMMathChain(llm=llm)

/Users/trucvietle/miniforge3/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [136]:
# initialize the math tool
math_tool = Tool(name='Calculator',
                 func=llm_math.run,
                 description='Useful for when you need to answer questions about math.')

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [137]:
# Load the prebuilt math tool instead
from langchain.agents import load_tools

tools = load_tools(['llm-math'], llm=llm)
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [138]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(agent="zero-shot-react-description",
                                   tools=tools, llm=llm, verbose=True,
                                   max_iterations=3)

**Zero-shot** means the agent functions on the current action only — it has no memory. It uses the **ReAct framework** to decide which tool to use, based solely on the tool’s description.

In [139]:
zero_shot_agent("What is (4.5*2.1)^2.2?")



> Entering new  chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'What is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [140]:
query = "If Mary has four apples and Joe brings two and a half apple boxes (each box contains eight apples), \
but half of the apples in the boxes are spoiled and thrown away. How many apples do we have in total?"

zero_shot_agent(query)



> Entering new  chain...
 I need to figure out how many apples are in the boxes and subtract the spoiled apples.
Action: Calculator
Action Input: 4 + (2.5 * 8)
Observation: Answer: 24.0
Thought: I need to subtract the spoiled apples.
Action: Calculator
Action Input: 24.0 - (2.5 * 4)
Observation: Answer: 14.0
Thought: I now know the final answer.
Final Answer: We have 14 apples in total.

> Finished chain.


{'input': 'If Mary has four apples and Joe brings two and a half apple boxes (apple box contains eight apples), but half of the apples in the boxes are spoiled and thrown away. How many apples do we have in total?',
 'output': 'We have 14 apples in total.'}

In [144]:
query = "Who is the Queen of Pop?"
# zero_shot_agent(query)

In [143]:
prompt = PromptTemplate(input_variables=["query"], template="{query}")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(name='Language Model',
                func=llm_chain.run,
                description='Use this tool for general purpose queries and logic')
tools.append(llm_tool)

# reinitialize the agent
zero_shot_agent = initialize_agent(agent="zero-shot-react-description",
                                   tools=tools, llm=llm, verbose=True,
                                   max_iterations=3)

In [145]:
zero_shot_agent(query)



> Entering new  chain...
 I need to find out who is considered the Queen of Pop.
Action: Language Model
Action Input: Who is the Queen of Pop?
Observation: 

The Queen of Pop is generally considered to be Madonna.
Thought: I now know the final answer.
Final Answer: Madonna is the Queen of Pop.

> Finished chain.


{'input': 'Who is the Queen of Pop?', 'output': 'Madonna is the Queen of Pop.'}

## Agent types

- **Zero-shot ReAct** -- the agent considers one single interaction and has no memory.
- **Conversational ReAct** -- same as Zero-shot ReAct but **with memory**.
- **ReAct Docstore** -- explicitly built for information retrieval using a LangChain docstore. This agent is memoryless.
- **Self-ask with Search** -- connect an LLM to a search engine (`SerpAPIWrapper`).

In [151]:
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(agent='conversational-react-description', 
                                        tools=tools, llm=llm, verbose=True,
                                        max_iterations=3, memory=memory)

# result = conversational_agent("What is the stock price of IBM on January 2nd")

In [152]:
# Using memory
# result = conversational_agent("What is the stock price of SAP on the same day?")

In [154]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())

tools = [Tool(name="Search", func=docstore.search, description='Search wikipedia'),
         Tool(name="Lookup", func=docstore.lookup, description='Lookup a term in wikipedia')
        ]

In [155]:
docstore_agent = initialize_agent(tools, llm, agent="react-docstore",
                                  verbose=True, max_iterations=3)

docstore_agent("What were Archimedes' last words?")



> Entering new  chain...
Thought: I need to search Archimedes and find his last words.
Action: Search[Archimedes]
Observation: Archimedes of Syracuse (; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of a spiral.Archimedes' other mathematical

{'input': "What were Archimedes' last words?",
 'output': '"Do not disturb my circles"'}

In [157]:
from langchain import SerpAPIWrapper

# initialize the search chain
search = SerpAPIWrapper(serpapi_api_key='serp_api_key')

# create a search tool
tools = [Tool(name="Intermediate Answer", func=search.run, description='Google search')]

# initialize the search enabled agent
self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

In [159]:
# self_ask_with_search("Who died younger: Plato, Socrates, or Aristotle?")